In [5]:
import pandas as pd
import csv
import os
import time
from collections import defaultdict
import numpy as np
import itertools
import db_structure
import db_structure_v2
import networkx as nx
from imp import reload
import matplotlib.pyplot as plt
import utilities as u
from flask import jsonify
import logging
# import exceptions
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from web.models import *

logging.basicConfig(format=' %(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
%load_ext autoreload
%aimport -web
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
draw_db = db

G = nx.DiGraph()
G.add_nodes_from(draw_db.table_names)
for table_name in draw_db.table_names:
    for sibling in draw_db.find_table_siblings(table_name):
        G.add_edge(table_name, sibling)
        G.add_edge(sibling, table_name)
    for child in draw_db.find_table_children(table_name):
        G.add_edge(table_name, child)
plt.figure(figsize=(10,10))

#draw with networkx built-ins
nx.draw_networkx(G, node_shape="None", width=0.2)

#OR

#draw using pygraphviz
#A = nx.nx_agraph.to_agraph(G)
#H = nx.nx_agraph.from_agraph(A)
#nx.draw_spring(H, node_shape="None", with_labels=True, width=0.2)

In [ ]:
x = db_structure_v2.DBMaker(dataset_name='TOPICC', directory_path='datasets\\TOPICC')

In [ ]:
x.create_db()

In [17]:
y = db_structure_v2.DBLinker(dataset_name='TOPICC')

In [18]:
y.remove_all_relationships()

In [19]:
y.add_global_fk('PudID')

In [6]:
z = db_structure_v2.DBExtractor(dataset_name='TOPICC')

In [7]:
z.get_joining_keys('CPR', 'asdf')

In [14]:
z.find_multi_tables_still_accessible_tables(['HOSPITALADMIT', 'HOSPITALADMIT'])

[]

In [9]:
z.find_paths_multi_tables(['CAREPROCESSES', 'DEATH', 'CARDIACSURGERY', 'CPR'])

[]

In [10]:
path = ['HOSPITALADMIT', 'CAREPROCESSES', 'DEATH']
start = time.time()
df = z.get_df_from_path(path, table_columns_of_interest=[('HOSPITALADMIT', 'PudID'), ('HOSPITALADMIT', 'Sex'), ('CAREPROCESSES', 'MechVent'), ('DEATH', 'DeathMode')])
end = time.time()
print(end-start)

0.19299983978271484


In [11]:
df

,HOSPITALADMIT_PudID,HOSPITALADMIT_Sex,CAREPROCESSES_MechVent,DEATH_DeathMode
0,27,Female,Yes,Withdrawal of care
1,78,Male,Yes,Withdrawal of care
2,125,Female,Yes,Failed resuscitation
3,154,Female,Yes,Failed resuscitation
4,155,Male,Yes,Withdrawal of care
...,...,...,...,...
270,9925,Female,Yes,Brain death
271,9932,Male,No,Withdrawal of care
272,9964,Male,Yes,Brain death
273,9990,Male,No,Withdrawal of care


In [ ]:
df['DeathMode'].value_counts()

In [15]:
filters = {
    'AdmitThroughED': {'type': 'list', 'filter': ['Yes', 'No']},
    'Sex': {'type': 'list', 'filter': ['Male', 'Female']}
}
new_df = z.aggregate_df(df, groupby_columns=['AdmitThroughED', 'Sex'], filters=filters)

[autoreload of web.routes failed: Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\extensions\autoreload.py", line 368, in superreload
    module = reload(module)
  File "C:\ProgramData\Anaconda3\lib\imp.py", line 315, in reload
    return importlib.reload(module)
  File "C:\ProgramData\Anaconda3\lib\importlib\__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 678, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "C:\Users\SShah50\Documents\Programming\cohort-visualization\web\routes.py", line 48, in <module>
    @flask_app.route('/login', methods=['GET', 'POST'])
  File "C:\Users\SShah50\AppData\Roaming\Python\Pyt

KeyError: 'AdmitThroughED'

In [ ]:
sorted(new_df.loc[:,'groupby_labels'].unique(), key=lambda x: x.upper())

In [ ]:
x, y = pd.cut(new_df['Count'], bins=2)

In [ ]:
for i in u.pairwise([1,2]):
    print(i)

In [ ]:
'3'.split('.')

In [ ]:
str((5, 3))

In [12]:
path = ['HOSPITALADMIT', 'CAREPROCESSES', 'PHYSIOSTATUS']
df = z.get_df_from_path(path, table_columns_of_interest=['HOSPITALADMIT.Sex', 'CAREPROCESSES.MechVent', 'PHYSIOSTATUS.LowpH'])

ValueError: too many values to unpack (expected 2)

In [ ]:
filters = {
    'MechVent': {'type': 'list', 'filter': ['Yes']},
    'LowpH': {'type': 'range', 'filter': {'min': 6.8, 'max': 6.9, 'bins': 4}},
    'Sex': None
}

z.aggregate_df(df, groupby_columns=['MechVent', 'LowpH'], filters=filters, aggregate_column='Sex')

In [ ]:
len(f)

In [ ]:
f.sort_values(by='LowpH')

In [ ]:
w = db_structure_v2.DBCustomizer(dataset_name='TOPICC')

In [ ]:
w.rename_column('HOSPITALADMIT', 'PudID', 'Patient ID')

In [ ]:
w.dump_customization()

In [ ]:
x = [(1, 2), (3, 4)]
for i, j in x:
    print(i)
    print(j)

In [ ]:
from web import db

In [ ]:
db.session.rollback()

In [ ]:
db.engine